# FIT5196 Assignment 1

#### Student Name:  Shihan Zhang

#### Student ID:        31268102

Date: 09/11/2020

Version: 3.0
- Version 1: process sinlge file & write to .xml file
- Version 2: added unicode Text processing
- Version 3: looping files(processing single file and write to file)

Environment: Python 3.6.0 and Anaconda 4.3.0 (64-bit)

## The workflow of this task:
Loop all the single source files

Within the loop we do the process 1 & 2
 1. process the single files
  - use regex to capture id, date, text
  - filter the none English tweets
 2. write to .xml file 

## Import Libraries

In [18]:
import os
import re
import langid

## Initialization the basic parameters
- savePath(path for write to file)
- xml format
- `allTweetDict` -- dictionary for store all tweets value

#### Structure of `allTweetDict`
 allTweetDict{
 - layer 1 key:   {date:'2020-03-22' 
   - layer 2 key:       {id:'1241576773457829889'
    - value:                     {tweet:'@TheDeshBhakt'}}}}

In [44]:
# output file save path
savedPath = '31268102.xml'

# xml format
xml_head = '''<?xml version="1.0" encoding="UTF-8"?>
<data>'''
xml_tail = '''\n</data>'''
xml_block = '''\n<tweets date="{date}">{tweetList}\n</tweets>'''
xml_tweet = '''\n<tweet id="{id}">{text}</tweet>'''

# Use dictionary format to store all data; xml can be converted with dictionary format;
allTweetDict = {}

### Create regex patterns
- `id_pattern` for capture id data `"id":"(\d*)"`
  - the regex `"id":`capture the string after "id:"
  - the regex `"()"`look for the group of the collection between `" "`
  - the regex `\d*` capture the zero of more digits, can also use `\d+`
  combine above regex together we got returned digits between `" "` and after the `"id:"`
  
- `text_pattern` for capture text data `"text":"([^"]*)"`
   - the regex `"text":`capture the string after "text:"
   - the regex `"()"`look for the group of the collection between `" "`
   - the regex `[^"]*` capture all the string before `"`, with `*` this condition can repret multiple times
   combine above regex together we got the return string(all type) between `" "` and after `"text:"`
   
- `date_pattern` for capture date data `"created_at":"(\d{4}-\d{2}-\d{2})`
   - the regex `"created_at:"`capture the string after "created:"
   - the regex `"()"`look for the group of the collection between `" "`
   - the regex `\d{4}-\d{2}-\d{2}` capture the digit with `-``
     - `\d{4}` digit repeat 4 times, `\d{2}` digit repreat 2 times
   combine above regex together we got the return date(4 digits after`-`, 2 digits after `-`, 2 digits after `-`) between `" "` and after `created_at:`

In [21]:
# id pattern
id_pattern = re.compile('\"id\":\"(\d*)\"')

# text_pattern
text_pattern = re.compile('\"text\":\"([^\"]*)\"')

# date_pattern, only need year-month-day so filter out the exact time.
date_pattern = re.compile('\"created_at\":\"(\d{4}-\d{2}-\d{2})')

## Load source file
- With `encoding = 'utf -8'` to reading all files in the folder
- Function for load source file. 
- Pass the 'filename' to function, function process the file, return the 'content' as the String value carry all text content of single file. 

In [22]:
def load_file(filename):
    with open(f"./31268102/{filename}", 'r', encoding='utf-8') as f:
        content = f.read()
    return content

## Process the text content
- This function is the main part of processing the tweets data.

#### Explanation of list length check for merging tweets data.
- To ensure that the three types of data (data, id, text) are merged in the correct position. Using `if-statement` to check based on the length of the list. If the list lengths of the three types of data are different, that measn there is data loss or index position mismatch. Then the tweets data will not be merged.

#### Unicode Text processing
- 1. Use`strip()` to detect the text, if tweet text only contain single`'\'` then skip this tweet. If the tweet has more than one backlash. Then use`rstrip()` to reformat to a single backslash.


#### Add new tweets to 'allTweetDict = {}'
In the loop of the index, we have the
- tweet_date as the `date_list[index]`,
- tweet_text as the `text_list[index]`,
- tweet_id as the `text_id[index]`

Then we check whether this 'tweet_date' is in the global dictionary or not.
- if the current date does exist in the dictionary:
  - we append the tweet text under this date (this is the under same date case)
- if not exist in the dictionary:
  - we check whether this is the unique id
  - only this id is unique in the dictionary
    - we insert the current date with the value(current id and text) to the dictionary

In [23]:
def process_text(content):
    # assign the dictionary as the global variable, to carry the value from all files data.
    global allTweetDict
    
    # list stores the values captured by regex
    id_list = re.findall(id_pattern, content)
    text_list = re.findall(text_pattern, content)
    date_list = re.findall(date_pattern, content)
    
    # remove duplicated date value
    unique_date = set(date_list)
    
    ## list length check for merging tweets data.
    if len(id_list) == len(text_list) == len(date_list):
        # looping the index range of data list
        for i in range(len(id_list)):
            tweet_date = date_list[i]
            tweet_id = id_list[i]
            tweet_text = '{}'.format(text_list[i])
            ## unicode Text processing
            if tweet_text.strip() == '\\':
                # jump to the next index if current text only contain single '\'
                return
            # remove the 'right' backslash
            tweet_text = tweet_text.rstrip('\\')
            
            # process the utf-16 surrogatespass
            second_text = eval(repr(tweet_text).replace('\\\\', '\\').replace('\\n\\n', '\\n'))
            res = second_text.encode('utf-16', 'surrogatepass').decode('utf-16')
            # reaplce the single backslash to white sapce
            res = res.replace('\\', '')

            # use langid to filter none English tweets
            if (langid.classify(res)[0]) == 'en':
                # insert to dictionary if tweets_date is new
                # prevent to insert dupilcated day content
                if tweet_date not in allTweetDict:
                    allTweetDict[tweet_date] = {tweet_id: res}
                
                # if tweets from different file has the same date
                else:
                    # assgin instant values(under date key) 
                    current_tweets = allTweetDict[tweet_date]
                    # check if tweet id already exist in the allTweetDict
                    # if not already in allTweetDict
                    # then insert this tweets text under the current date with current id 
                    if tweet_id not in current_tweets:
                        allTweetDict[tweet_date][tweet_id] = res
            # else:
            #     print NONE English tweets
            #     print('====', tweet_text)
    
    # if list length check not pass
    else:
        print("lenght of lists are not equal")

## Write to .xml file
#### with following functions
- `write_head()`
 - write xml format header tags
 
- `write_file()`
 - wrtie tweets text content
 
- `write_tail()`
 - write xml format footer tags

##### Why use write function separately?
Based on the overall program structure, I need to continuously write content to the txt file. 
But I only need to write the header tag and footer tag once. 
So I decided to make header and footer separate functions and call them only once.
Between header and footer, I can repeatedly call the function to writing tweets.

In [24]:
def write_head():
    with open(savedPath, 'w', encoding='utf-8') as f:
        f.write(xml_head)

In [25]:
def write_file(blockText):
    # write to .xml file
    with open(savedPath, 'a', encoding='utf-8') as f:
        f.write(blockText)

In [26]:
def write_tail():
    with open(savedPath, 'a', encoding='utf-8') as f:
        f.write(xml_tail)

## Main function 
to Combine all sub-functions and process all files using loop
#### wrtie main tweets content
- First layer, use loop the dictionary with k,v
  - use list `each_block` to reformat the tweets' content in second layer.
  
- Second layer, loop the tweets'text and id under the date
  - use the `xml format` string to combine the data with xml tags.
      - then append to `each_block` list
  - join the elements in the `each_block` list by the tweets'date.
      - then format the joined data with xml tags
  - write to file

In [27]:
def main():
    # append all file under the dirctory to folder list
    folders = os.listdir(r'.\31268102')

    # loop all the files
    for index, fileName in enumerate(folders):
        if index > 3:
            break
        # print('=========', fileName, '====================')
        # process the single file
        originText = load_file(fileName)
        process_text(originText)

    # writting file
    # write header tags
    write_head()
    
    # wrtie main tweets content
    for date, tweetList in allTweetDict.items():
        each_block = []
        for tweet_id, tweet_text in tweetList.items():
            each_block.append(
                xml_tweet.format(id=tweet_id, text=tweet_text)
            )
        date_block = xml_block.format(date=date, tweetList=''.join(each_block))
        write_file(date_block)
    
    # wrtie footer tags
    write_tail()

In [30]:
if __name__ == '__main__':
    main()